# Parsing DAIDE strings

In [6]:
from daidepp import create_daide_grammar, daide_visitor
from pprint import pprint

In [8]:
grammar = create_daide_grammar(level=130)

## Successful parses

In [18]:
def parse(grammar, message):
    parse_tree = grammar.parse(message)
    output = daide_visitor.visit(parse_tree)
    pprint(output)

In [19]:
message = 'PRP (PCE (AUS GER FRA))'
parse(grammar, message)

('PRP', ('PCE', ['AUS', 'GER', 'FRA']))


In [20]:
message = 'PRP (AND (SLO (ENG)) (SLO (GER)) (SLO (RUS)) (AND (SLO (ENG)) (SLO (GER)) (SLO (RUS))))'
parse(grammar, message)

('PRP',
 ('AND',
  [('SLO', 'ENG'),
   ('SLO', 'GER'),
   ('SLO', 'RUS'),
   ('AND', [('SLO', 'ENG'), ('SLO', 'GER'), ('SLO', 'RUS')])]))


In [21]:
message = 'PRP (AND (SLO (ENG)) (SLO (GER)) (SLO (RUS)) (AND (SLO (ENG)) (SLO (GER)) (ORR (PCE (AUS GER)) (ALY (GER FRA) VSS (TUR RUS)))))'
parse(grammar, message)

('PRP',
 ('AND',
  [('SLO', 'ENG'),
   ('SLO', 'GER'),
   ('SLO', 'RUS'),
   ('AND',
    [('SLO', 'ENG'),
     ('SLO', 'GER'),
     ('ORR',
      [('PCE', ['AUS', 'GER']),
       ('ALY_VSS', (['GER', 'FRA'], ['TUR', 'RUS']))])])]))


In [37]:
grammar_20 = create_daide_grammar(level=20, allow_just_arrangment=True)
message = 'XDO ((ENG AMY LVP) HLD)'
parse(grammar_20, message)

('XDO', ('HLD', ('ENG', 'AMY', 'LVP')))


## Invalid DAIDE messages

### Non-daide token

In [22]:
message = 'PRP (PCE (AuUS GER FRA))'
parse(grammar, message)

ParseError: Rule 'power' didn't match at 'AuUS GER FRA))' (line 1, column 11).

### Daide token in higher level

In [28]:
grammar_10 = create_daide_grammar(level=10)
message = 'PRP (XDO ((ENG AMY LVP) HLD))'
parse(grammar_10, message)

ParseError: Rule 'arrangement' didn't match at 'XDO ((ENG AMY LVP) H' (line 1, column 6).

In [41]:
grammar_10 = create_daide_grammar(level=10)
message = 'PRP (XDO ((ENG AMY LVP) HLD))'
parse(grammar_10, message)

ParseError: Rule 'arrangement' didn't match at 'XDO ((ENG AMY LVP) H' (line 1, column 6).

### Invalid string structure

In [43]:
message = 'PRP (PCE ((ENG AMY LVP) HLD))'
parse(grammar, message)

ParseError: Rule 'power' didn't match at '(ENG AMY LVP) HLD))' (line 1, column 11).

# Constructing DAIDE strings with keyword objects

In [44]:
from daidepp.keywords import *

In [55]:
print(PCE("AUS", "ENG", "FRA"))

PCE ( AUS ENG FRA )


In [56]:
print(ALYVSS(["AUS", "GER"], ["ENG", "FRA"]))

ALY ( AUS GER ) VSS ( ENG FRA )


In [59]:
print(IFF(PCE("AUS", "ENG"), PRP(PCE("AUS", "ITA"))))

IFF ( PCE ( AUS ENG ) ) THN ( PRP ( PCE ( AUS ITA ) ) )


In [58]:
print(IFF(PCE("AUS", "ENG"), PRP(PCE("AUS", "ITA")), PRP(PCE("GER", "FRA"))))

IFF ( PCE ( AUS ENG ) ) THN ( PRP ( PCE ( AUS ITA ) ) ) ELS ( PRP ( PCE ( GER FRA ) ) )


In [69]:
from dataclasses import dataclass
from typing_extensions import Literal

POWER = Literal["AUS", "ENG", "FRA", "GER", "ITA", "RUS", "TUR"]

@dataclass
class PCE:
    powers: List[POWER]

    def __init__(self, *powers):
        self.powers = powers

    def __str__(self):
        return "PCE ( " + " ".join(self.powers) + " )"
    
    def as_tuple(self):
        return ('PCE', ('AUS', 'GER', 'FRA'))
    

pce = PCE("AUS", "GER")
print(pce.powers)
print(pce.tuple)

('AUS', 'GER')


AttributeError: 'PCE' object has no attribute 'tuple'